# Set the working directory to the src folder on your machine

In [1]:
import os
os.chdir(r"C:\Users\osmou\Documents\GitHub\wingspan-bird-brain\src")
print(os.getcwd())

C:\Users\osmou\Documents\GitHub\wingspan-bird-brain\src


In [2]:
%load_ext autoreload
%autoreload 2

In [188]:
from env import WingspanEnvironment
from Gameplay_Constants import BIRDFEEDER_FACES, FOOD_TYPES, MAX_NUMBER_DIE_IN_BIRDFEEDER
from Birdfeeder import Birdfeeder
import gymnasium
from gymnasium.spaces import Discrete, MultiDiscrete, Box
import numpy as np
from pettingzoo.test import parallel_api_test

In [191]:
env = WingspanEnvironment(render_mode="human", log=False)
parallel_api_test(env, num_cycles=1)

NotImplementedError: 

In [187]:
# birdfeeder = Birdfeeder()

# # Defined for each agent in reset()
# action_space = gymnasium.spaces.Dict({
#     "Gain Food" : Discrete(len(FOOD_TYPES)),  # Gain food one at a time
# })

# # Defined for each agent in reset()
# observation_space = gymnasium.spaces.Dict({
#     "observations": gymnasium.spaces.Dict({
#         "Birdfeeder" : MultiDiscrete([len(BIRDFEEDER_FACES)+1,]*MAX_NUMBER_DIE_IN_BIRDFEEDER), # +1 for die out of feeder
#     }),
#     "action_mask": gymnasium.spaces.Dict({
#         "Gain Food": Box(low=0, high=1, shape=(len(FOOD_TYPES),), dtype="int8")
#     })
# })

# # created in observe() fn
# observation = {
#     # 0=Fish, 1=Rodent, 2=Fruit, 3=Invertebrate, 4=Seed, 5=Invertebrate+seed, 6=dieOutOfFeeder
#     "Birdfeeder": [
#         np.argwhere(np.array(BIRDFEEDER_FACES)==birdfeeder.food[i])[0][0] if i < len(birdfeeder.food) else 6
#         for i in range(MAX_NUMBER_DIE_IN_BIRDFEEDER)
#     ]
# }
# action_mask = {
#     "Gain Food": np.array([1 if f in str(birdfeeder.food) else 0 for f in FOOD_TYPES], dtype="int8")
# }

# # Sample to choose action
# sample = action_space.sample(action_mask)

# print(birdfeeder)
# print("FOOD_TYPES:", FOOD_TYPES)
# print(action_mask)
# print("Sample:", sample)
# print("Chosen food:", BIRDFEEDER_FACES[sample["Gain Food"]])